Import Libraries

In [27]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly.express as px
import plotly.graph_objects as go
import plotly
plotly.offline.init_notebook_mode(connected = True)

Load Datasets

In [3]:
items_cats = pd.read_csv('data/item_categories.csv')
items = pd.read_csv('data/items.csv')
sales_train = pd.read_csv('data/sales_train.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
shops = pd.read_csv('data/shops.csv')
test = pd.read_csv('data/test.csv')

In [7]:
sales_train.head()
test.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [8]:
items_cats.head()
items.head()
shops.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


Will need to merge several tables into one for easier reference

In [9]:
sales_item = sales_train.merge(items, on='item_id', how='left') # gains item_name and cat_id per item_id on the train set
sales_item_shops = sales_item.merge(shops, on='shop_id', how='left') # gains shop name per shop_id on the train set
df = sales_item_shops.merge(items_cats, on='item_category_id', how='left') # gain item_cat_name per cat_id on the train set
df.sample(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
287318,19.03.2013,2,23,6740,499.00,1.0,"StarCraft II: Wings of Liberty [PC, Jewel, рус...",30,"Москва ТК ""Буденовский"" (пав.А2)",Игры PC - Стандартные издания
1803929,30.07.2014,18,31,19004,229.00,1.0,СОФИЯ ПРЕКРАСНАЯ:КАК СТАТЬ ПРИНЦЕССОЙ,40,"Москва ТЦ ""Семеновский""",Кино - DVD
2353295,08.01.2015,24,24,5821,799.00,1.0,PlayStation Plus 3-месячная подписка: Карта оп...,35,"Москва ТК ""Буденовский"" (пав.К7)",Карты оплаты - PSN
2092505,16.10.2014,21,31,13058,149.00,2.0,КУХНЯ В ПАРИЖЕ (регион),40,"Москва ТЦ ""Семеновский""",Кино - DVD
2624834,15.05.2015,28,7,9405,299.00,2.0,Брелок металлический WOT Счастливчик (100111),70,"Воронеж ТРЦ ""Максимир""",Подарки - Сувениры (в навеску)
2074350,30.10.2014,21,57,10383,399.00,1.0,ГЕРАКЛ: Начало легенды,40,"Якутск Орджоникидзе, 56",Кино - DVD
2750869,15.07.2015,30,52,18831,149.00,1.0,"СМЕШАРИКИ НОВЫЕ ПРИКЛЮЧЕНИЯ Выпуск 5, 6",40,"Уфа ТК ""Центральный""",Кино - DVD
2574131,03.04.2015,27,25,8765,269.16,1.0,БЕЛОСНЕЖКА И ОХОТНИК (BD),37,"Москва ТРК ""Атриум""",Кино - Blu-Ray
969527,23.10.2013,9,50,3686,899.00,1.0,"God of War 3 (Essentials) [PS3, русская версия]",19,"Тюмень ТЦ ""Гудвин""",Игры - PS3
1527581,29.03.2014,14,44,4233,1699.00,1.0,"Killzone: Наемник [PS Vita, русская версия]",22,"Самара ТЦ ""Мелодия""",Игры - PSVita


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   date_block_num      int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_day        float64
 6   item_name           object 
 7   item_category_id    int64  
 8   shop_name           object 
 9   item_category_name  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 246.4+ MB


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
date_block_num,2935849.0,14.569911,9.422988,0.0,7.0,14.0,23.0,33.0
shop_id,2935849.0,33.001728,16.226973,0.0,22.0,31.0,47.0,59.0
item_id,2935849.0,10197.227057,6324.297354,0.0,4476.0,9343.0,15684.0,22169.0
item_price,2935849.0,890.853233,1729.799631,-1.0,249.0,399.0,999.0,307980.0
item_cnt_day,2935849.0,1.242641,2.618834,-22.0,1.0,1.0,1.0,2169.0
item_category_id,2935849.0,40.001383,17.100759,0.0,28.0,40.0,55.0,83.0


Here we can see anomalies in the data: 
* a negative value for the price 
* a negative quantity of goods sold per day.

In [16]:
print('Count of rows with abnormal price: {:.0f}'.format(len(df[df['item_price'] <= 0])))
print('Count of rows with abnormal quantity of goods sold per day: {:.0f}'.format(len(df[df['item_cnt_day'] < 0])))

Count of rows with abnormal price: 1
Count of rows with abnormal quantity of goods sold per day: 7356


In [18]:
print('Fraction of Anomalies in Dataset: ',7357/2935849*100,'%')

Fraction of Anomalies in Dataset:  0.25059190714508817 %


Remove all rows with anomalies.

In [19]:
df = df[~(df['item_price'] <= 0) & ~(df['item_cnt_day'] < 0)]
df.describe(include = ['object']).T

,count,unique,top,freq
date,2928492,1034,28.12.2013,9415
item_name,2928492,21804,Фирменный пакет майка 1С Интерес белый (34*42)...,31337
shop_name,2928492,60,"Москва ТЦ ""Семеновский""",235185
item_category_name,2928492,84,Кино - DVD,563938


In [20]:
print(f'Count duplicates: {df.duplicated().sum()}')
df = df.drop_duplicates()

Count duplicates: 6


In [24]:
df.sample(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
1589170,2014-01-01,15,16,7019,2299.0,2.0,"Titanfall [Xbox 360, русская версия]",23,"Коломна ТЦ ""Рио""",Игры - XBOX 360
1714230,2014-01-01,17,25,12033,99.0,1.0,ИРОНИЯ СУДЬБЫ ИЛИ С ЛЕГКИМ ПАРОМ (регион),40,"Москва ТРК ""Атриум""",Кино - DVD
334176,2013-01-01,2,59,2808,999.0,1.0,"Diablo III [PC, Jewel, русская версия]",30,"Ярославль ТЦ ""Альтаир""",Игры PC - Стандартные издания
161312,2013-01-01,1,21,12085,199.0,1.0,ИСХОДНЫЙ КОД (BD),37,"Москва МТРЦ ""Афи Молл""",Кино - Blu-Ray
1760086,2014-01-01,17,59,2442,999.0,1.0,"Crysis 3 (Classics) [Xbox 360, русская версия]",23,"Ярославль ТЦ ""Альтаир""",Игры - XBOX 360


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2928486 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   date_block_num      int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_day        float64
 6   item_name           object 
 7   item_category_id    int64  
 8   shop_name           object 
 9   item_category_name  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 245.8+ MB


In [22]:
df['date'] = df['date'].astype('datetime64[Y]')

print('Min date:', df['date'].min().date())
print('Max date', df['date'].max().date())

Min date: 2013-01-01
Max date 2015-01-01


In [31]:
df_cnt_item = df.groupby('date_block_num').agg({'item_cnt_day':'sum'}).reset_index().rename(columns={'date_block_num':'number_of_month', 'item_cnt_day':'item_cnt'})
fig = px.line(df_cnt_item, x="number_of_month", y="item_cnt")
fig.update_layout(
    title='The number of goods sold on a monthly basis',
    xaxis_title='Time',
    yaxis_title='Sales')
fig.show()